In [ ]:
import pandas as pd
import re
import openai
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import random
import tabulate
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import (mean_squared_error, mean_absolute_error, r2_score, 
                             confusion_matrix, accuracy_score, precision_score, recall_score, f1_score)
from tqdm.auto import tqdm
import joblib
from itertools import product
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split

# === Load data ===
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

# === Feature and Target ===
X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

y_scores_df = pd.DataFrame(
    df_processed["scores"].tolist(),
    columns=[
        "openai/gpt-4o",
        "anthropic/claude-3.5-sonnet",
        "deepseek/deepseek-chat",
        "perplexity/sonar"
    ]
)
Y = y_scores_df.values
category_labels = df_processed["Source"].astype("category").cat.codes

# === Train/Test Split ===
X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm,
    Y,
    test_size=0.20,
    stratify=category_labels,
    random_state=42
)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

# === Define all regression models ===
regression_models = {
    "Random Forest": MultiOutputRegressor(RandomForestRegressor(
        n_estimators=200, max_depth=25, min_samples_leaf=5, random_state=42, n_jobs=-1)),
    
    "XGBoost": MultiOutputRegressor(XGBRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.1, objective="reg:squarederror", random_state=42, n_jobs=-1)),
    
    "MLP": MultiOutputRegressor(MLPRegressor(
        hidden_layer_sizes=(256, 128), activation='relu', solver='adam', max_iter=300, random_state=42)),
    
    "Ridge": MultiOutputRegressor(Ridge(alpha=1.0))
}

# === Run training and evaluation ===
results = {}

for name, model in regression_models.items():
    print(f"\n=== Training {name} ===")
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    # === Per-LLM MSE ===
    per_llm_mse = [
        mean_squared_error(Y_test[:, i], Y_pred[:, i])
        for i in range(Y.shape[1])
    ]

    # === Top-1 Accuracy ===
    true_best = np.argmax(Y_test, axis=1)
    pred_best = np.argmax(Y_pred, axis=1)
    top1_acc = accuracy_score(true_best, pred_best)

    # === Top-2 Inclusion (Hit@2) ===
    sorted_pred = np.argsort(Y_pred, axis=1)[:, ::-1]  # descending
    top2_pred = [set(row[:2]) for row in sorted_pred]
    hit_at_2 = np.array([
        true_best[i] in top2_pred[i] for i in range(len(true_best))
    ], dtype=int)
    hit_at_2_acc = np.mean(hit_at_2)

    # === Save results ===
    results[name] = {
        "model": model,
        "Y_pred": Y_pred,
        "MSE_per_LLM": per_llm_mse,
        "Top1 Accuracy": top1_acc,
        "Top2 Inclusion": hit_at_2_acc
    }

    # === Print basic results ===
    print(f"Per-LLM MSE: {per_llm_mse}")
    print(f"Top-1 Accuracy: {top1_acc:.4f}")
    print(f"Top-2 Inclusion: {hit_at_2_acc:.4f}")


# Report True Scores (All 4 LLMs, from Y_test)

In [ ]:
# Sort ground-truth scores across all prompts (full dataset)
Y_full_sorted = np.sort(Y, axis=1)[:, ::-1]

# Extract per-rank scores across full dataset
rank1_all = Y_full_sorted[:, 0]
rank2_all = Y_full_sorted[:, 1]
rank3_all = Y_full_sorted[:, 2]
rank4_all = Y_full_sorted[:, 3]

# Print statistics
print("=== Ranked Score Statistics Across Full Dataset (All Prompts) ===")
print(f"Rank 1: Mean={np.mean(rank1_all):.4f}, Min={np.min(rank1_all):.4f}, Max={np.max(rank1_all):.4f}")
print(f"Rank 2: Mean={np.mean(rank2_all):.4f}, Min={np.min(rank2_all):.4f}, Max={np.max(rank2_all):.4f}")
print(f"Rank 3: Mean={np.mean(rank3_all):.4f}, Min={np.min(rank3_all):.4f}, Max={np.max(rank3_all):.4f}")
print(f"Rank 4: Mean={np.mean(rank4_all):.4f}, Min={np.min(rank4_all):.4f}, Max={np.max(rank4_all):.4f}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set consistent styling
sns.set(style="whitegrid", font_scale=1.4)

plt.figure(figsize=(5.5, 4))  # Match previous plot dimensions
sns.boxplot(
    data=[rank1_all, rank2_all, rank3_all, rank4_all],
    width=0.4,
    linewidth=1.4
)

# Axis labels and ticks
plt.xticks(
    range(4),
    ['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4'],
    fontsize=12
)
plt.yticks([0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40], fontsize=12)
plt.ylabel("Normalized Score", fontsize=14)
plt.xlabel("", fontsize=14)

# Optional horizontal gridlines
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.ylim(0.08, 0.42)
plt.tight_layout()

# Save to file
plt.savefig("score_distributions.pdf", format="pdf", bbox_inches="tight")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Clean renaming
y_scores_df_clean = y_scores_df.rename(columns={
    "openai/gpt-4o": "GPT-4o",
    "anthropic/claude-3.5-sonnet": "Claude 3.5 Sonnet",
    "deepseek/deepseek-chat": "DeepSeek Chat",
    "perplexity/sonar": "Perplexity Sonar"
})

# Melt for seaborn
y_melted = y_scores_df_clean.melt(var_name="LLM", value_name="Normalized Score")

# Plot
plt.figure(figsize=(5.5, 4))  # narrower width to pull boxes closer
sns.set(style="whitegrid", font_scale=1.4)

sns.violinplot(
    x="LLM", 
    y="Normalized Score", 
    data=y_melted,
    inner="box", 
    cut=0,
    linewidth=1.4,
    scale='area'
)

# Grid and ticks
plt.grid(axis='y', linestyle='--', alpha=0.5)
plt.xticks(rotation=10, fontsize=12)
plt.yticks([0.10, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40], fontsize=12)
plt.xlabel("")
plt.ylabel("Normalized Score", fontsize=14)
plt.ylim(0.08, 0.42)
plt.tight_layout()

# Save figure
plt.savefig("fig_violin_per_llm_clean.pdf")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Predicted values from Random Forest
Y_pred_rf = results["Random Forest"]["Y_pred"]

# Sorted scores descending
sorted_preds = np.sort(Y_pred_rf, axis=1)[:, ::-1]
top1, top2, top3 = sorted_preds[:, 0], sorted_preds[:, 1], sorted_preds[:, 2]

gap_1_2 = top1 - top2
gap_1_3 = top1 - top3

# Shared bin edges for consistent x-axis
gap_min = min(gap_1_2.min(), gap_1_3.min())
gap_max = max(gap_1_2.max(), gap_1_3.max())
bins = np.linspace(gap_min, gap_max, 31)  # 30 bins

# Compute histogram values to match y-axis
counts_1_2, _ = np.histogram(gap_1_2, bins=bins)
counts_1_3, _ = np.histogram(gap_1_3, bins=bins)
max_freq = max(counts_1_2.max(), counts_1_3.max())

plt.rcParams.update({
    "axes.titlesize": 16,
    "axes.labelsize": 14,
    "xtick.labelsize": 12,
    "ytick.labelsize": 12
})

# Histogram for Rank-1 vs Rank-2
plt.figure(figsize=(6, 4))
plt.hist(gap_1_2, bins=bins, edgecolor='black', color='skyblue')
plt.xlabel("Score Gap")
plt.ylabel("Frequency")
plt.ylim(0, max_freq + 5)
plt.tight_layout()
plt.savefig("fig_gap_rank12.pdf", format="pdf", bbox_inches="tight")

# Histogram for Rank-1 vs Rank-3
plt.figure(figsize=(6, 4))
plt.hist(gap_1_3, bins=bins, edgecolor='black', color='salmon')
plt.xlabel("Score Gap")
plt.ylabel("Frequency")
plt.ylim(0, max_freq + 5)
plt.tight_layout()
plt.savefig("fig_gap_rank13.pdf", format="pdf", bbox_inches="tight")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Get true scores
Y_true = Y  # already normalized scores, shape = (N, 4)

# Compute sorted true scores (descending)
sorted_true_scores = np.sort(Y_true, axis=1)[:, ::-1]  # shape = (N, 4)
rank1 = sorted_true_scores[:, 0]
rank2 = sorted_true_scores[:, 1]
rank3 = sorted_true_scores[:, 2]
rank4 = sorted_true_scores[:, 3]

# === 1. Boxplot: Score Distributions by Rank ===
plt.figure(figsize=(8, 5))
plt.boxplot([rank1, rank2, rank3, rank4], labels=["Rank 1", "Rank 2", "Rank 3", "Rank 4"])
plt.title("Boxplot of True Scores by Rank")
plt.ylabel("Score")
plt.grid(axis='y')
plt.tight_layout()
plt.show()

# === 2. Histograms (Same y-axis for visual comparison) ===
# Compute common y-limit
counts_r1, _ = np.histogram(rank1, bins=30)
counts_r2, _ = np.histogram(rank2, bins=30)
counts_r3, _ = np.histogram(rank3, bins=30)
counts_r4, _ = np.histogram(rank4, bins=30)
max_freq_all = max(counts_r1.max(), counts_r2.max(), counts_r3.max(), counts_r4.max())

# Plot histogram for each rank
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

ranks = [rank1, rank2, rank3, rank4]
titles = ["Rank 1", "Rank 2", "Rank 3", "Rank 4"]
colors = ['green', 'blue', 'orange', 'red']

for i, ax in enumerate(axes.flatten()):
    ax.hist(ranks[i], bins=30, edgecolor='black', color=colors[i])
    ax.set_title(f"Score Distribution: {titles[i]}")
    ax.set_xlabel("Score")
    ax.set_ylabel("Frequency")
    ax.set_ylim(0, max_freq_all + 5)
    ax.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Predicted ranking indices from RF model
sorted_pred_idx = np.argsort(results["Random Forest"]["Y_pred"], axis=1)[:, ::-1]

# Extract true scores of predicted top-1 to top-4
true_scores_top1 = [Y_test[i, sorted_pred_idx[i][0]] for i in range(len(Y_test))]
true_scores_top2 = [Y_test[i, sorted_pred_idx[i][1]] for i in range(len(Y_test))]
true_scores_top3 = [Y_test[i, sorted_pred_idx[i][2]] for i in range(len(Y_test))]
true_scores_top4 = [Y_test[i, sorted_pred_idx[i][3]] for i in range(len(Y_test))]

# Report statistics
print("=== True Scores for Top Ranked Predicted Models (Random Forest) ===")
print(f"Top-1: Mean={np.mean(true_scores_top1):.4f}, Min={np.min(true_scores_top1):.4f}, Max={np.max(true_scores_top1):.4f}")
print(f"Top-2: Mean={np.mean(true_scores_top2):.4f}, Min={np.min(true_scores_top2):.4f}, Max={np.max(true_scores_top2):.4f}")
print(f"Top-3: Mean={np.mean(true_scores_top3):.4f}, Min={np.min(true_scores_top3):.4f}, Max={np.max(true_scores_top3):.4f}")
print(f"Top-4: Mean={np.mean(true_scores_top4):.4f}, Min={np.min(true_scores_top4):.4f}, Max={np.max(true_scores_top4):.4f}")



In [ ]:
import matplotlib.pyplot as plt

# Choose best model result from earlier block
best_model_name = "Random Forest"  # or "Ridge" depending on what you analyze
Y_pred = results[best_model_name]["Y_pred"]

# === Score Gap Analysis ===
sorted_preds = np.sort(Y_pred, axis=1)
top1_vals = sorted_preds[:, -1]
top2_vals = sorted_preds[:, -2]
gaps = top1_vals - top2_vals

# Plot
plt.figure(figsize=(6, 4))
plt.hist(gaps, bins=30, edgecolor="black")
plt.title(f"{best_model_name} — Score Gap (Top1 − Top2)")
plt.xlabel("Score Gap")
plt.ylabel("Frequency")
plt.tight_layout()
plt.show()

# Stats
print(f"\n=== {best_model_name} Score Gap Stats ===")
print(f"Mean gap     = {np.mean(gaps):.4f}")
print(f"Median gap   = {np.median(gaps):.4f}")
print(f"Std. dev gap = {np.std(gaps):.4f}")

# === Top-2 Ranking Consistency ===
true_sorted = np.argsort(Y_test, axis=1)[:, ::-1]
pred_sorted = np.argsort(Y_pred, axis=1)[:, ::-1]

def top2_match(true_row, pred_row):
    return len(set(true_row[:2]) & set(pred_row[:2])) / 2.0

top2_scores = np.array([
    top2_match(true_sorted[i], pred_sorted[i])
    for i in range(len(Y_test))
])

mean_top2_match = np.mean(top2_scores)

print(f"\n=== Top-2 Ranking Consistency (set overlap) ===")
print(f"Mean Top-2 Match Score: {mean_top2_match:.4f}")


In [ ]:
from sklearn.metrics import accuracy_score

# True best LLM for each test sample
true_best = np.argmax(Y_test, axis=1)

# Sorted predicted indices and scores
pred_scores = results["Random Forest"]["Y_pred"]
sorted_preds_idx = np.argsort(pred_scores, axis=1)[:, ::-1]
sorted_preds_val = np.sort(pred_scores, axis=1)[:, ::-1]
top1 = sorted_preds_idx[:, 0]
top2 = sorted_preds_idx[:, 1]
gap = sorted_preds_val[:, 0] - sorted_preds_val[:, 1]

# Thresholds to test
thresholds = [0.01, 0.03, 0.05, 0.08, 0.10]
print("=== Confidence-Aware Fallback Results (Random Forest) ===")
print(f"{'Threshold':<10} {'Acc':<6} {'Fallback%':<10} {'Avg LLMs Queried':<18}")

for t in thresholds:
    fallback_mask = gap < t  # trigger fallback
    acc = []
    num_llms = []

    for i in range(len(true_best)):
        if fallback_mask[i]:
            # Try top-1 and top-2 → correct if true_best in either
            if true_best[i] in [top1[i], top2[i]]:
                acc.append(1)
            else:
                acc.append(0)
            num_llms.append(2)
        else:
            # Route only to top-1
            acc.append(int(true_best[i] == top1[i]))
            num_llms.append(1)

    acc_val = np.mean(acc)
    fallback_rate = np.mean(fallback_mask)
    avg_llms = np.mean(num_llms)

    print(f"{t:<10} {acc_val:.4f}   {fallback_rate:.2f}       {avg_llms:.2f}")


In [ ]:
# Recompute if needed
Y_pred_rf = results["Random Forest"]["Y_pred"]
sorted_pred_vals = np.sort(Y_pred_rf, axis=1)
top1_vals = sorted_pred_vals[:, -1]
top2_vals = sorted_pred_vals[:, -2]
gaps = top1_vals - top2_vals

# Plot
plt.figure(figsize=(6, 4))
plt.hist(gaps, bins=30, edgecolor="black")
plt.title("Random Forest – Score Gap (Top-1 − Top-2)")
plt.xlabel("Score Gap")
plt.ylabel("Number of Prompts")
plt.tight_layout()
plt.show()


In [ ]:
print("\n=== Regression Summary Table ===")
print(f"{'Model':<15} {'Top-1 Acc':<10} {'Top-2 Inc':<10} {'Avg MSE':<10}")
for name, data in results.items():
    mse_avg = np.mean(data["MSE_per_LLM"])
    print(f"{name:<15} {data['Top1 Accuracy']:<10.4f} {data['Top2 Inclusion']:<10.4f} {mse_avg:<10.4f}")



In [ ]:
import matplotlib.pyplot as plt

model_names = list(results.keys())
top1_accuracies = [results[m]['Top1 Accuracy'] for m in model_names]
top2_inclusions = [results[m]['Top2 Inclusion'] for m in model_names]

x = np.arange(len(model_names))
width = 0.35

plt.figure(figsize=(8, 5))
plt.bar(x - width/2, top1_accuracies, width, label='Top-1 Accuracy')
plt.bar(x + width/2, top2_inclusions, width, label='Top-2 Inclusion')

plt.ylabel("Accuracy")
plt.title("Top-1 and Top-2 Routing Accuracy by Regression Model")
plt.xticks(x, model_names, rotation=15)
plt.ylim(0, 1.0)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

# === Define all regression models ===
regression_models = {
    "Random Forest": MultiOutputRegressor(RandomForestRegressor(
        n_estimators=200, max_depth=25, min_samples_leaf=5, random_state=42, n_jobs=-1)),
    
    "XGBoost": MultiOutputRegressor(XGBRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.1, objective="reg:squarederror", random_state=42, n_jobs=-1)),
    
    "MLP": MultiOutputRegressor(MLPRegressor(
        hidden_layer_sizes=(256, 128), activation='relu', solver='adam', max_iter=300, random_state=42)),
    
    "Ridge": MultiOutputRegressor(Ridge(alpha=1.0)),

    "SVR": MultiOutputRegressor(SVR(kernel='rbf', C=1.0, epsilon=0.1))
}

# === Run training and evaluation ===
results = {}

for name, model in regression_models.items():
    print(f"\n=== Training {name} ===")
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    # === Per-LLM MSE ===
    per_llm_mse = [
        mean_squared_error(Y_test[:, i], Y_pred[:, i])
        for i in range(Y.shape[1])
    ]

    # === Top-1 Accuracy ===
    true_best = np.argmax(Y_test, axis=1)
    pred_best = np.argmax(Y_pred, axis=1)
    top1_acc = accuracy_score(true_best, pred_best)

    # === Top-2 Inclusion (Hit@2) ===
    sorted_pred = np.argsort(Y_pred, axis=1)[:, ::-1]  # descending
    top2_pred = [set(row[:2]) for row in sorted_pred]
    hit_at_2 = np.array([
        true_best[i] in top2_pred[i] for i in range(len(true_best))
    ], dtype=int)
    hit_at_2_acc = np.mean(hit_at_2)

    # === Save results ===
    results[name] = {
        "model": model,
        "Y_pred": Y_pred,
        "MSE_per_LLM": per_llm_mse,
        "Top1 Accuracy": top1_acc,
        "Top2 Inclusion": hit_at_2_acc
    }

    # === Print basic results ===
    print(f"Per-LLM MSE: {per_llm_mse}")
    print(f"Top-1 Accuracy: {top1_acc:.4f}")
    print(f"Top-2 Inclusion: {hit_at_2_acc:.4f}")


In [ ]:
# ------------------------------------------------------------
# 1. Imports
# ------------------------------------------------------------
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
import numpy as np

# ------------------------------------------------------------
# 2. Define the five lightweight regressors
# ------------------------------------------------------------
regression_models = {
    "Random Forest": MultiOutputRegressor(
        RandomForestRegressor(
            n_estimators=200, max_depth=25, min_samples_leaf=5,
            random_state=42, n_jobs=-1)
    ),
    "XGBoost": MultiOutputRegressor(
        XGBRegressor(
            n_estimators=200, max_depth=6, learning_rate=0.1,
            objective="reg:squarederror", random_state=42, n_jobs=-1)
    ),
    "MLP": MultiOutputRegressor(
        MLPRegressor(
            hidden_layer_sizes=(256, 128), activation='relu',
            solver='adam', max_iter=300, random_state=42)
    ),
    "Ridge": MultiOutputRegressor(Ridge(alpha=1.0)),
    "SVR": MultiOutputRegressor(SVR(kernel='rbf', C=1.0, epsilon=0.1))
}

# ------------------------------------------------------------
# 3. Train, predict, and store results
# ------------------------------------------------------------
results = {}
for name, model in regression_models.items():
    print(f"\n=== Training {name} ===")
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    results[name] = {"model": model, "Y_pred": Y_pred}

# ------------------------------------------------------------
# 4. Correct win-rate computation
#    Win rate = (0.5 * N_spec + N_better) / total
#    where
#      N_spec   = router actually chose the reference LLM
#      N_better = router chose a *different* LLM whose score > reference LLM
# ------------------------------------------------------------
def win_rate_against_llms(Y_test, Y_pred):
    total = Y_test.shape[0]
    pick = np.argmax(Y_pred, axis=1)                       # index router picked
    chosen_scores = Y_test[np.arange(total), pick]         # score of router pick
    win_rates = []
    for llm_idx in range(Y_test.shape[1]):
        spec_mask   = (pick == llm_idx)                    # router chose that LLM
        n_spec      = np.sum(spec_mask)
        llm_scores  = Y_test[:, llm_idx]                   # ground-truth scores of that LLM
        better_mask = ~spec_mask & (chosen_scores > llm_scores)
        n_better    = np.sum(better_mask)
        win_rate = (0.5 * n_spec + n_better) / total
        win_rates.append(win_rate)
    return win_rates

# ------------------------------------------------------------
# 5. Print win-rate table
# ------------------------------------------------------------
print("\n=== Win Rates per LLM (correct formula) ===")
headers = ["Model", "GPT-4o", "Claude", "DeepSeek", "Perplexity"]
print(f"{headers[0]:<18} {headers[1]:>10} {headers[2]:>10} {headers[3]:>10} {headers[4]:>12}")

for name, info in results.items():
    wr = win_rate_against_llms(Y_test, info["Y_pred"])
    wr_pct = [f"{100*r:.2f}%" for r in wr]
    print(f"{name:<18} {wr_pct[0]:>10} {wr_pct[1]:>10} {wr_pct[2]:>10} {wr_pct[3]:>12}")


In [ ]:
# =======================================================
# 0.  Imports
# =======================================================
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

# -------------------------------------------------------
# 1.  Train / test split  (skip if you already have X_train...)
# -------------------------------------------------------
#   X : (N, d)  prompt embeddings
#   Y : (N, 4)  PairRanker scores for [gpt-4o, claude-3.5, deepseek, perplexity]
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42, stratify=np.argmax(Y, axis=1)
)

# -------------------------------------------------------
# 2.  Define lightweight regressors
# -------------------------------------------------------
regression_models = {
    "Random Forest": MultiOutputRegressor(
        RandomForestRegressor(
            n_estimators=200, max_depth=25, min_samples_leaf=5,
            random_state=42, n_jobs=-1
        )
    ),
    "Ridge": MultiOutputRegressor(Ridge(alpha=1.0)),
    "XGBoost": MultiOutputRegressor(
        XGBRegressor(
            n_estimators=200, max_depth=6, learning_rate=0.1,
            objective="reg:squarederror", random_state=42, n_jobs=-1
        )
    ),
    "SVR": MultiOutputRegressor(SVR(kernel="rbf", C=1.0, epsilon=0.1)),
    "MLP": MultiOutputRegressor(
        MLPRegressor(
            hidden_layer_sizes=(256, 128), activation="relu",
            solver="adam", max_iter=300, random_state=42
        )
    ),
}

# -------------------------------------------------------
# 3.  Train, predict, evaluate
# -------------------------------------------------------
results = {}
true_best = np.argmax(Y_test, axis=1)

for name, model in regression_models.items():
    print(f"\n=== Training {name} ===")
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    # per-LLM MSE
    per_llm_mse = [
        mean_squared_error(Y_test[:, i], Y_pred[:, i])
        for i in range(Y.shape[1])
    ]

    # Top-1 accuracy
    pred_best = np.argmax(Y_pred, axis=1)
    top1_acc = accuracy_score(true_best, pred_best)

    # Hit@2
    top2_pred = np.argsort(Y_pred, axis=1)[:, ::-1][:, :2]
    hit2 = [true_best[i] in top2_pred[i] for i in range(len(true_best))]
    hit2_acc = np.mean(hit2)

    # store
    results[name] = {
        "model": model,
        "Y_pred": Y_pred,
        "MSE_per_LLM": per_llm_mse,
        "Top1 Accuracy": top1_acc,
        "Hit@2": hit2_acc,
    }

    # quick printout
    print(f"Per-LLM MSE : {[f'{m:.4f}' for m in per_llm_mse]}")
    print(f"Top-1 Acc   : {top1_acc:.4f}")
    print(f"Hit@2       : {hit2_acc:.4f}")

# -------------------------------------------------------
# 4.  Grouped bar chart: Top-1 vs Hit@2
# -------------------------------------------------------
regressor_names = list(results.keys())
top1_vals = [results[r]["Top1 Accuracy"] for r in regressor_names]
hit2_vals = [results[r]["Hit@2"] for r in regressor_names]

# sort by descending Top-1 for nicer layout
order = np.argsort(top1_vals)[::-1]
regressor_names = [regressor_names[i] for i in order]
top1_vals = [top1_vals[i] for i in order]
hit2_vals = [hit2_vals[i] for i in order]

x = np.arange(len(regressor_names))
width = 0.35

plt.figure(figsize=(9, 4.5))
bars1 = plt.bar(x - width / 2, top1_vals, width, label="Top-1")
bars2 = plt.bar(x + width / 2, hit2_vals, width, label="Hit@2")

plt.ylabel("Fraction of prompts")
plt.title("Overall Routing Performance")
plt.xticks(x, regressor_names, rotation=15)
plt.ylim(0, 1.0)
plt.legend()

# annotate bars
for bar in list(bars1) + list(bars2):
    h = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, h + 0.02,
             f"{h:.2f}", ha="center", va="bottom", fontsize=9)

plt.tight_layout()
plt.show()


In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import accuracy_score

# === Load and prepare data ===
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

# Features and labels
X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

y_scores_df = pd.DataFrame(
    df_processed["scores"].tolist(),
    columns=[
        "openai/gpt-4o",
        "anthropic/claude-3.5-sonnet",
        "deepseek/deepseek-chat",
        "perplexity/sonar"
    ]
)
Y = y_scores_df.values
category_labels = df_processed["Source"].astype("category").cat.codes

# Train-test split (stratified by Source)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm,
    Y,
    test_size=0.20,
    stratify=category_labels,
    random_state=42
)

# === Train the Random Forest regression model ===
rf = MultiOutputRegressor(RandomForestRegressor(
    n_estimators=200, max_depth=25, min_samples_leaf=5, random_state=42, n_jobs=-1
))
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)

# === Confidence-Aware Fallback Analysis ===
true_top1 = np.argmax(Y_test, axis=1)
sorted_preds = np.sort(Y_pred, axis=1)[:, ::-1]
ranked_preds = np.argsort(Y_pred, axis=1)[:, ::-1]

top1_idx = ranked_preds[:, 0]
top2_idx = ranked_preds[:, 1]
top1_score = sorted_preds[:, 0]
top2_score = sorted_preds[:, 1]

# Compute confidence gap
gap = top1_score - top2_score

# Thresholds to test
thresholds = np.arange(0.01, 0.21, 0.01)
fallback_stats = []

for tau in thresholds:
    fallback_mask = gap < tau
    routed_llms = np.where(
        fallback_mask[:, None],
        np.stack([top1_idx, top2_idx], axis=1),
        np.stack([top1_idx, top1_idx], axis=1)  # duplicate if not falling back
    )

    # Accuracy check: top-1 in ground truth within routed
    correct = [(true_top1[i] in routed_llms[i]) for i in range(len(true_top1))]
    accuracy = np.mean(correct)
    fallback_pct = np.mean(fallback_mask)
    avg_llms_queried = 1 + fallback_pct

    fallback_stats.append((tau, accuracy, fallback_pct, avg_llms_queried))

# Convert to DataFrame
fallback_df = pd.DataFrame(
    fallback_stats,
    columns=["Threshold", "Accuracy", "Fallback Rate", "Avg. LLMs Queried"]
)

# Print results
print("\n=== Confidence-Aware Fallback Results (Random Forest) ===")
print(fallback_df.to_string(index=False))


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.plot(fallback_df["Avg. LLMs Queried"],
         fallback_df["Accuracy"]*100,
         marker="o")
for _, row in fallback_df.iterrows():
    if row["Threshold"] in {0.03,0.05,0.10}:
        plt.annotate(f"τ={row['Threshold']:.2f}",
                     (row["Avg. LLMs Queried"], row["Accuracy"]*100),
                     textcoords="offset points", xytext=(0,6), ha='center')
plt.xlabel("Average LLMs Queried")
plt.ylabel("Accuracy (%)")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.savefig("acc_vs_cost_rf.pdf")


In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# === Load enhanced pairwise dataset ===
with open("df_pairwise_v2.pkl", "rb") as f:
    df_pairwise = pickle.load(f)

# === Extract prompt embeddings (numerical)
X_embed = np.vstack(df_pairwise["embedding"].values)

# === One-hot encode llm_A, llm_B, and Source (categorical)
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(df_pairwise[["llm_A", "llm_B", "Source"]])

# === Final input: [embedding + one-hot categorical features]
X = np.hstack([X_embed, X_cat])

# === Labels
y = df_pairwise["label"].values

# === Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Train Random Forest
rf = RandomForestClassifier(n_estimators=200, max_depth=25, random_state=42)
rf.fit(X_train, y_train)

# === Evaluate
y_pred = rf.predict(X_test)
acc = accuracy_score(y_test, y_pred)

print(f"\n✅ Realistic Random Forest Accuracy (no leakage): {acc:.4f}")


In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier

# === Load dataset ===
with open("df_pairwise_v2.pkl", "rb") as f:
    df_pairwise = pickle.load(f)

# === Extract prompt embeddings
X_embed = np.vstack(df_pairwise["embedding"].values)

# === One-hot encode llm_A and llm_B
encoder = OneHotEncoder(sparse_output=False)
X_cat = encoder.fit_transform(df_pairwise[["llm_A", "llm_B"]])

# === Final feature matrix
X = np.hstack([X_embed, X_cat])
y = df_pairwise["label"].values

# === Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Define classifiers
classifiers = {
    "Random Forest": RandomForestClassifier(n_estimators=200, max_depth=25, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=200, max_depth=6, learning_rate=0.1, use_label_encoder=False, eval_metric='logloss', random_state=42),
    "MLP": MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=300, random_state=42),
    "Ridge Classifier": RidgeClassifier(),
    "SVC": SVC(kernel='rbf', probability=True, random_state=42)
}

# === Evaluate each model
print(f"{'Model':20s}  Acc   Prec  Recall   F1     AUC")
print("-" * 60)

for name, model in classifiers.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    if hasattr(model, "predict_proba"):
        y_prob = model.predict_proba(X_test)[:, 1]
    else:
        # fallback for models without predict_proba
        y_prob = y_pred

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f"{name:20s}  {acc:.3f}  {prec:.3f}  {rec:.3f}   {f1:.3f}  {auc:.3f}")


In [ ]:
import pickle, itertools, numpy as np, pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor

# ------------------------------------------------------------------
# 0 · Assumed pre-computed objects
#     X_train, Y_train, X_test, Y_test from your earlier split
#     llm_cols list (length 4) with fixed order
# ------------------------------------------------------------------

# ------------------------------------------------------------------
# 1 · (ROUTER) Train multi-output regressors  -----------------------
# ------------------------------------------------------------------
regression_models = {
    "Random Forest": MultiOutputRegressor(
        RandomForestRegressor(n_estimators=200, max_depth=25,
                              min_samples_leaf=5, random_state=42, n_jobs=-1))
}

router_results = {}
for name, model in regression_models.items():
    print(f"\n=== Training router: {name} ===")
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    true_best = np.argmax(Y_test, axis=1)
    pred_best = np.argmax(Y_pred, axis=1)
    top1_acc  = accuracy_score(true_best, pred_best)

    # Hit@2 (Top-2 inclusion)
    sorted_pred = np.argsort(Y_pred, axis=1)[:, ::-1]
    hit2 = [true_best[i] in sorted_pred[i, :2] for i in range(len(true_best))]
    hit2_acc = np.mean(hit2)

    router_results[name] = dict(model=model,
                                Y_pred=Y_pred,
                                Top1=top1_acc,
                                Hit2=hit2_acc)
    print(f"Top-1: {top1_acc:.4f}  ·  Hit@2: {hit2_acc:.4f}")

# Choose the Random Forest router for the rest
rf_router = router_results["Random Forest"]["model"]
Y_pred_rf = router_results["Random Forest"]["Y_pred"]

# ------------------------------------------------------------------
# 2 · (SELECTOR) Load pair-wise dataset and train classifier --------
# ------------------------------------------------------------------
pair_df = pd.read_pickle("pairwise_llm_dataset_v2.pkl")

# Extract features
emb_mat   = np.vstack(pair_df["embedding"].values)          # (M, 1536)
llm_a_id  = pair_df["LLM_A_id"].values.reshape(-1, 1)
llm_b_id  = pair_df["LLM_B_id"].values.reshape(-1, 1)

# One-hot encode A and B IDs (4 classes each)
enc = OneHotEncoder(sparse_output=False)
llm_onehot = enc.fit_transform(np.hstack([llm_a_id, llm_b_id]))  # (M, 8)

X_pair = np.hstack([emb_mat, llm_onehot])   # final feat dim = 1536 + 8
y_pair = pair_df["label"].values

# Split
Xp_tr, Xp_te, yp_tr, yp_te = train_test_split(
        X_pair, y_pair, test_size=0.2, random_state=42, stratify=y_pair)

# Train lightweight selector (LogReg)
selector = LogisticRegression(max_iter=1000, class_weight="balanced")
selector.fit(Xp_tr, yp_tr)

print("\n=== Selector validation metrics ===")
print(classification_report(yp_te, selector.predict(Xp_te), digits=4))

# ------------------------------------------------------------------
# 3 · Pipeline evaluation (router + selector) ----------------------
# ------------------------------------------------------------------
def pipeline_decision(emb, pred_scores, tau=0.05):
    order = np.argsort(pred_scores)[::-1]
    top1, top2 = order[:2]
    gap = pred_scores[top1] - pred_scores[top2]

    if gap >= tau:
        return top1                       # single query case
    # build selector feature
    pair_feat = np.hstack([
        emb,
        enc.transform([[top1, top2]])[0]  # one-hot A,B
    ])
    return top1 if selector.predict(pair_feat.reshape(1, -1))[0] == 1 else top2

tau = 0.05
final_preds = []
for i in range(len(X_test)):
    best_idx = pipeline_decision(X_test[i], Y_pred_rf[i], tau)
    final_preds.append(best_idx)

final_acc = accuracy_score(np.argmax(Y_test, axis=1), final_preds)

print("\n=== Pipeline results (τ = 0.05) ===")
print(f"Router Top-1: {router_results['Random Forest']['Top1']:.4f}")
print(f"Router Hit@2: {router_results['Random Forest']['Hit2']:.4f}")
print(f"Pipeline Top-1 (after selector): {final_acc:.4f}")


In [ ]:
# ================================================================
# 0 · Imports
# ================================================================
import pickle, itertools, joblib
import numpy as np, pandas as pd
from sklearn.preprocessing import normalize, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor

# ================================================================
# 1 · Load df_processed and build primary matrices
# ================================================================
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

# prompt embeddings
X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, axis=1)                         # (N, 1536)

# true score matrix
llm_cols = [
    "openai/gpt-4o",
    "anthropic/claude-3.5-sonnet",
    "deepseek/deepseek-chat",
    "perplexity/sonar"
]
Y = pd.DataFrame(df_processed["scores"].tolist(),
                 columns=llm_cols).values             # (N, 4)

# ================================================================
# 2 · Train/test split for router (stratify if 'Source' exists)
# ================================================================
if "Source" in df_processed.columns:
    stratify_arg = df_processed["Source"].astype("category").cat.codes
else:
    print("Warning: 'Source' column not found — using unstratified split.")
    stratify_arg = None

X_tr_r, X_te_r, Y_tr_r, Y_te_r = train_test_split(
    X_norm, Y, test_size=0.20, random_state=42, stratify=stratify_arg)

# ================================================================
# 3 · ROUTER — Random-Forest multi-output regressor
# ================================================================
rf_router = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=200,
        max_depth=25,
        min_samples_leaf=5,
        n_jobs=-1,
        random_state=42))
rf_router.fit(X_tr_r, Y_tr_r)
Y_pred_r = rf_router.predict(X_te_r)

# Router metrics
true_best = np.argmax(Y_te_r, axis=1)
pred_best = np.argmax(Y_pred_r, axis=1)
router_top1 = accuracy_score(true_best, pred_best)
hit2 = [
    true_best[i] in np.argsort(Y_pred_r[i])[::-1][:2]
    for i in range(len(true_best))
]
router_hit2 = np.mean(hit2)

print("\n=== ROUTER RESULTS ===")
print(f"Top-1 accuracy : {router_top1:.4f}")
print(f"Hit@2          : {router_hit2:.4f}")

# ================================================================
# 4 · Build pair-wise dataset (skip ties) for selector
# ================================================================
pair_rows = []
for emb_i, scores_i in zip(X_norm, Y):
    for a, b in itertools.combinations(range(4), 2):
        if np.isclose(scores_i[a], scores_i[b]):       # skip exact ties
            continue
        winner, loser = (a, b) if scores_i[a] > scores_i[b] else (b, a)

        # winner row  (label 1)
        pair_rows.append((emb_i, winner, loser, 1))
        # loser row   (label 0)
        pair_rows.append((emb_i, loser, winner, 0))

pair_df = pd.DataFrame(pair_rows,
                       columns=["embedding", "LLM_A", "LLM_B", "label"])

# ================================================================
# 5 · FEATURES for selector  (embedding + one-hot IDs)
# ================================================================
emb_mat = np.vstack(pair_df["embedding"].values)
enc = OneHotEncoder(sparse_output=False)
llm_onehot = enc.fit_transform(pair_df[["LLM_A", "LLM_B"]].values)

scaler = StandardScaler()
emb_scaled = scaler.fit_transform(emb_mat)

X_pair = np.hstack([emb_scaled, llm_onehot]).astype(np.float32)
y_pair = pair_df["label"].values

Xp_tr, Xp_te, yp_tr, yp_te = train_test_split(
    X_pair, y_pair, test_size=0.20, random_state=42, stratify=y_pair)

# ================================================================
# 6 · SELECTOR — Random-Forest classifier
# ================================================================
rf_selector = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,
    min_samples_leaf=2,
    n_jobs=-1,
    class_weight="balanced",
    random_state=42)
rf_selector.fit(Xp_tr, yp_tr)

print("\n=== SELECTOR VALIDATION METRICS ===")
print(classification_report(yp_te, rf_selector.predict(Xp_te), digits=4))

# ================================================================
# 7 · PIPELINE EVALUATION  (τ = 0.08)
# ================================================================
tau = 0.08
final_preds = []

for emb, scores_pred, true_idx in zip(X_te_r, Y_pred_r, true_best):
    order = np.argsort(scores_pred)[::-1]
    top1, top2 = order[:2]
    gap = scores_pred[top1] - scores_pred[top2]

    if gap >= tau:
        final_preds.append(top1)                   # single query
    else:
        # build selector input: scaled embedding + one-hot pair
        feat = np.hstack([
            scaler.transform(emb.reshape(1, -1)),
            enc.transform([[top1, top2]])
        ])
        choose_A = rf_selector.predict(feat)[0]    # 1 ⇒ A wins
        final_preds.append(top1 if choose_A else top2)

pipeline_top1 = accuracy_score(true_best, final_preds)

print("\n=== PIPELINE RESULTS  (τ = 0.08) ===")
print(f"Router Top-1         : {router_top1:.4f}")
print(f"Router Hit@2         : {router_hit2:.4f}")
print(f"Pipeline Top-1       : {pipeline_top1:.4f}")

# ================================================================
# 8 · Save models and encoders (optional)
# ================================================================
joblib.dump(rf_router,   "rf_router.pkl")
joblib.dump(rf_selector, "rf_selector.pkl")
joblib.dump(enc,         "pair_onehot_encoder.pkl")
joblib.dump(scaler,      "embedding_scaler.pkl")
print("\nSaved: rf_router.pkl, rf_selector.pkl, pair_onehot_encoder.pkl, embedding_scaler.pkl")


In [ ]:
# ================================================================
# 0 · Imports
# ================================================================
import pickle, itertools, joblib
import numpy as np, pandas as pd
from sklearn.preprocessing import normalize, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor

# ================================================================
# 1 · ROUTER DATA  (load embeddings + scores; stratify by Source)
# ================================================================
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

# embeddings
X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

# score matrix (4 LLMs)
llm_cols = [
    "openai/gpt-4o",
    "anthropic/claude-3.5-sonnet",
    "deepseek/deepseek-chat",
    "perplexity/sonar"
]
Y = pd.DataFrame(df_processed["scores"].tolist(), columns=llm_cols).values

# stratified split by Source (if available)
category_labels = df_processed["Source"].astype("category").cat.codes
X_tr_r, X_te_r, Y_tr_r, Y_te_r = train_test_split(
    X_norm, Y, test_size=0.20, stratify=category_labels, random_state=42)

# ================================================================
# 2 · ROUTER — Random-Forest multi-output regressor
# ================================================================
rf_router = MultiOutputRegressor(
    RandomForestRegressor(
        n_estimators=200, max_depth=25,
        min_samples_leaf=5, n_jobs=-1, random_state=42)
)
rf_router.fit(X_tr_r, Y_tr_r)
Y_pred_r = rf_router.predict(X_te_r)

# Router metrics
true_best = np.argmax(Y_te_r, axis=1)
pred_best = np.argmax(Y_pred_r, axis=1)
router_top1 = accuracy_score(true_best, pred_best)
hit2 = [true_best[i] in np.argsort(Y_pred_r[i])[::-1][:2]
        for i in range(len(true_best))]
router_hit2 = np.mean(hit2)

print("\n=== ROUTER RESULTS ===")
print(f"Top-1 accuracy : {router_top1:.4f}")
print(f"Hit@2          : {router_hit2:.4f}")

# ================================================================
# 3 · SELECTOR DATA  (load pair-wise dataset)
# ================================================================
pair_df = pd.read_pickle("pairwise_llm_dataset_v2.pkl")

# build features: embedding + one-hot(LLM_A, LLM_B)
emb_mat = np.vstack(pair_df["embedding"].values)
enc = OneHotEncoder(sparse_output=False)
llm_pair_1hot = enc.fit_transform(pair_df[["LLM_A_id", "LLM_B_id"]].values)

scaler = StandardScaler()
emb_scaled = scaler.fit_transform(emb_mat)

X_pair = np.hstack([emb_scaled, llm_pair_1hot]).astype(np.float32)
y_pair = pair_df["label"].values

Xp_tr, Xp_te, yp_tr, yp_te = train_test_split(
    X_pair, y_pair, test_size=0.20, random_state=42, stratify=y_pair)

# ================================================================
# 4 · SELECTOR — Random-Forest binary classifier
# ================================================================
rf_selector = RandomForestClassifier(
    n_estimators=300, max_depth=None,
    min_samples_leaf=2, class_weight="balanced",
    n_jobs=-1, random_state=42)
rf_selector.fit(Xp_tr, yp_tr)

print("\n=== SELECTOR VALIDATION METRICS ===")
print(classification_report(yp_te, rf_selector.predict(Xp_te), digits=4))

# ================================================================
# 5 · PIPELINE EVALUATION (τ = 0.08)
# ================================================================
tau = 0.08
final_preds = []

for emb, scores_pred in zip(X_te_r, Y_pred_r):
    order = np.argsort(scores_pred)[::-1]
    top1, top2 = order[:2]
    gap = scores_pred[top1] - scores_pred[top2]

    if gap >= tau:
        final_preds.append(top1)                    # single query case
    else:
        # selector feature: scaled embedding ++ one-hot pair
        feat = np.hstack([
            scaler.transform(emb.reshape(1, -1)),
            enc.transform([[top1, top2]])
        ])
        choose_A = rf_selector.predict(feat)[0]     # 1 ⇒ A wins
        final_preds.append(top1 if choose_A else top2)

pipeline_top1 = accuracy_score(true_best, final_preds)

print("\n=== PIPELINE RESULTS (τ = 0.08) ===")
print(f"Router Top-1        : {router_top1:.4f}")
print(f"Router Hit@2        : {router_hit2:.4f}")
print(f"Pipeline Top-1      : {pipeline_top1:.4f}")

# ================================================================
# 6 · (Optional) save models and encoders
# ================================================================
joblib.dump(rf_router,   "rf_router.pkl")
joblib.dump(rf_selector, "rf_selector.pkl")
joblib.dump(enc,         "pair_onehot_encoder.pkl")
joblib.dump(scaler,      "embedding_scaler.pkl")
print("\nSaved models and encoders to disk.")


In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

# === Load and prepare your data ===
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

y_scores_df = pd.DataFrame(
    df_processed["scores"].tolist(),
    columns=[
        "openai/gpt-4o",
        "anthropic/claude-3.5-sonnet",
        "deepseek/deepseek-chat",
        "perplexity/sonar"
    ]
)
Y = y_scores_df.values
category_labels = df_processed["Source"].astype("category").cat.codes

X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm,
    Y,
    test_size=0.20,
    stratify=category_labels,
    random_state=42
)

# === Define regression models ===
regression_models = {
    "Random Forest": MultiOutputRegressor(RandomForestRegressor(
        n_estimators=200, max_depth=25, min_samples_leaf=5, random_state=42, n_jobs=-1)),
    "XGBoost": MultiOutputRegressor(XGBRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.1,
        objective="reg:squarederror", random_state=42, n_jobs=-1)),
    "MLP": MultiOutputRegressor(MLPRegressor(
        hidden_layer_sizes=(256, 128), activation='relu',
        solver='adam', max_iter=300, random_state=42)),
    "Ridge": MultiOutputRegressor(Ridge(alpha=1.0)),
    "SVR": MultiOutputRegressor(SVR(kernel='rbf', C=1.0, epsilon=0.1))
}

results = {}

# === Train, predict, and evaluate ===
for name, model in regression_models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    # 1) Per-LLM MSE
    per_llm_mse = [
        mean_squared_error(Y_test[:, i], Y_pred[:, i])
        for i in range(Y.shape[1])
    ]

    # 2) Top-1 Accuracy
    true_best = np.argmax(Y_test, axis=1)
    pred_best = np.argmax(Y_pred, axis=1)
    top1_acc = accuracy_score(true_best, pred_best)

    # 3) Hit-@-2 Accuracy
    #    Check if the true best index is in the model's top two predictions
    sorted_idxs = np.argsort(Y_pred, axis=1)[:, ::-1]  # descending order
    top2_sets = [set(row[:2]) for row in sorted_idxs]
    hit2_acc = np.mean([true_best[i] in top2_sets[i] for i in range(len(true_best))])

    # === Save results ===
    results[name] = {
        "MSE_per_LLM": per_llm_mse,
        "Top1 Accuracy": top1_acc,
        "Hit@2 Accuracy": hit2_acc
    }

    # === Print results ===
    print(f"Per-LLM MSE: {per_llm_mse}")
    print(f"Top-1 Accuracy: {top1_acc:.4f}")
    print(f"Hit@2 Accuracy: {hit2_acc:.4f}")


In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, accuracy_score

# === Load and prepare data ===
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

y_scores_df = pd.DataFrame(
    df_processed["scores"].tolist(),
    columns=[
        "openai/gpt-4o",
        "anthropic/claude-3.5-sonnet",
        "deepseek/deepseek-chat",
        "perplexity/sonar"
    ]
)
Y = y_scores_df.values
category_labels = df_processed["Source"].astype("category").cat.codes

X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm,
    Y,
    test_size=0.20,
    stratify=category_labels,
    random_state=42
)

# === Define regression models ===
regression_models = {
    "Random Forest": MultiOutputRegressor(RandomForestRegressor(
        n_estimators=200, max_depth=25, min_samples_leaf=5,
        random_state=42, n_jobs=-1)),
    "XGBoost": MultiOutputRegressor(XGBRegressor(
        n_estimators=200, max_depth=6, learning_rate=0.1,
        objective="reg:squarederror", random_state=42, n_jobs=-1)),
    "MLP": MultiOutputRegressor(MLPRegressor(
        hidden_layer_sizes=(256, 128), activation='relu',
        solver='adam', max_iter=300, random_state=42)),
    "Ridge": MultiOutputRegressor(Ridge(alpha=1.0)),
    "SVR": MultiOutputRegressor(SVR(kernel='rbf', C=1.0, epsilon=0.1))
}

results = {}

# === Train, predict, and evaluate ===
for name, model in regression_models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)

    # Per-LLM MSE
    per_llm_mse = [
        mean_squared_error(Y_test[:, i], Y_pred[:, i])
        for i in range(Y.shape[1])
    ]

    # True-best and true-second indices per example
    sorted_true_idxs = np.argsort(Y_test, axis=1)[:, ::-1]  # descending by true score
    true_best    = sorted_true_idxs[:, 0]
    true_second  = sorted_true_idxs[:, 1]

    # Model's top-1 prediction index per example
    pred_best = np.argmax(Y_pred, axis=1)

    # Top-1 Accuracy
    top1_acc = np.mean(pred_best == true_best)

    # Top-1-or-2 Accuracy:
    #  counts as correct if pred_best matches either true_best or true_second
    top1_or2_acc = np.mean([
        pred_best[i] in {true_best[i], true_second[i]}
        for i in range(len(pred_best))
    ])

    # Save & print
    results[name] = {
        "MSE_per_LLM": per_llm_mse,
        "Top1 Accuracy": top1_acc,
        "Top1-or-2 Accuracy": top1_or2_acc
    }

    print(f"Per-LLM MSE:       {per_llm_mse}")
    print(f"Top-1 Accuracy:    {top1_acc:.4f}")
    print(f"Top-1-or-2 Accuracy: {top1_or2_acc:.4f}")


In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

# === 1. Load and prepare the test data ===
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

# Features and labels
X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

llm_keys = [
    "openai/gpt-4o",
    "anthropic/claude-3.5-sonnet",
    "deepseek/deepseek-chat",
    "perplexity/sonar"
]
y_scores_df = pd.DataFrame(
    df_processed["scores"].tolist(),
    columns=llm_keys
)
Y = y_scores_df.values
category_labels = df_processed["Source"].astype("category").cat.codes

# Train-test split (stratified by Source)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm, Y, test_size=0.20, stratify=category_labels, random_state=42
)

# === 2. Train the Random Forest regression model (Router) ===
rf = MultiOutputRegressor(RandomForestRegressor(
    n_estimators=200, max_depth=25, min_samples_leaf=5, random_state=42, n_jobs=-1
))
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)

# === 3. Load or train MLP binary classifier ===
with open("df_pairwise_v2.pkl", "rb") as f:
    df_pairwise = pickle.load(f)

X_embed_cls = np.vstack(df_pairwise["embedding"].values)
encoder_cls = OneHotEncoder(sparse_output=False)
X_cat_cls = encoder_cls.fit_transform(df_pairwise[["llm_A", "llm_B"]])
X_cls = np.hstack([X_embed_cls, X_cat_cls])
y_cls = df_pairwise["label"].values

X_train_cls, X_val_cls, y_train_cls, y_val_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42
)
mlp = MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=300, random_state=42)
mlp.fit(X_train_cls, y_train_cls)

# === 4. Confidence-based routing logic with correct mapping ===
true_top1 = np.argmax(Y_test, axis=1)
sorted_preds = np.sort(Y_pred, axis=1)[:, ::-1]
ranked_preds = np.argsort(Y_pred, axis=1)[:, ::-1]

top1_idx = ranked_preds[:, 0]
top2_idx = ranked_preds[:, 1]
top1_score = sorted_preds[:, 0]
top2_score = sorted_preds[:, 1]
gap = top1_score - top2_score

# Mapping from full LLM names to short names
key_to_short = {
    "openai/gpt-4o": "gpt-4o",
    "anthropic/claude-3.5-sonnet": "claude",
    "deepseek/deepseek-chat": "deepseek",
    "perplexity/sonar": "perplexity"
}

thresholds = np.arange(0.01, 0.21, 0.01)
results = []

for tau in thresholds:
    routed_set_contains_best = []
    overall_selection_correct = []

    for i in range(len(X_test)):
        emb = X_test[i]
        # Always include top1, sometimes top2 if gap < tau
        routed_llms = [top1_idx[i]]
        use_classifier = False

        if gap[i] < tau:
            routed_llms.append(top2_idx[i])
            use_classifier = True

        # Coverage: does the routed set include the true best LLM?
        routed_set_contains_best.append(true_top1[i] in routed_llms)

        # Final pick:
        if not use_classifier:
            selected_idx = top1_idx[i]
        else:
            llmA_idx, llmB_idx = top1_idx[i], top2_idx[i]
            llmA_full = list(y_scores_df.columns)[llmA_idx]
            llmB_full = list(y_scores_df.columns)[llmB_idx]
            llmA_name = key_to_short[llmA_full]
            llmB_name = key_to_short[llmB_full]
            pair = pd.DataFrame([[llmA_name, llmB_name]], columns=["llm_A", "llm_B"])
            pair_cat = encoder_cls.transform(pair)
            mlp_input = np.hstack([emb, pair_cat[0]])
            prob = mlp.predict_proba([mlp_input])[0][1]
            selected_idx = llmA_idx if prob >= 0.5 else llmB_idx

        overall_selection_correct.append(selected_idx == true_top1[i])

    routed_set_acc = np.mean(routed_set_contains_best)
    overall_acc = np.mean(overall_selection_correct)
    results.append((tau, routed_set_acc, overall_acc))

# Convert to DataFrame
results_df = pd.DataFrame(
    results,
    columns=["Threshold", "Coverage Accuracy", "Overall Selection Accuracy"]
)
print(results_df)

# === 5. Plotting ===
plt.figure(figsize=(7, 4))
plt.plot(results_df["Threshold"], results_df["Coverage Accuracy"], marker='o', label="Coverage Accuracy")
plt.plot(results_df["Threshold"], results_df["Overall Selection Accuracy"], marker='s', label="Overall Selection Accuracy")
plt.xlabel("Gap Threshold ($\\tau$)")
plt.ylabel("Accuracy")
plt.title("Confidence-Based Routing: Coverage and Selection Accuracy vs. Threshold")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# ------------------------------------------------------------
# 1. Imports
# ------------------------------------------------------------
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPClassifier

# ------------------------------------------------------------
# 2. Load and preprocess data
# ------------------------------------------------------------
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

llm_keys = [
    "openai/gpt-4o",
    "anthropic/claude-3.5-sonnet",
    "deepseek/deepseek-chat",
    "perplexity/sonar"
]
y_scores_df = pd.DataFrame(
    df_processed["scores"].tolist(),
    columns=llm_keys
)
Y = y_scores_df.values
category_labels = df_processed["Source"].astype("category").cat.codes

X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm, Y, test_size=0.20, stratify=category_labels, random_state=42
)

# ------------------------------------------------------------
# 3. Train Random Forest Regressor (Router)
# ------------------------------------------------------------
rf = MultiOutputRegressor(RandomForestRegressor(
    n_estimators=200, max_depth=25, min_samples_leaf=5, random_state=42, n_jobs=-1
))
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)

# ------------------------------------------------------------
# 4. Train or Load MLP Classifier
# ------------------------------------------------------------
with open("df_pairwise_v2.pkl", "rb") as f:
    df_pairwise = pickle.load(f)

X_embed_cls = np.vstack(df_pairwise["embedding"].values)
encoder_cls = OneHotEncoder(sparse_output=False)
X_cat_cls = encoder_cls.fit_transform(df_pairwise[["llm_A", "llm_B"]])
X_cls = np.hstack([X_embed_cls, X_cat_cls])
y_cls = df_pairwise["label"].values

X_train_cls, X_val_cls, y_train_cls, y_val_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42
)
mlp = MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=300, random_state=42)
mlp.fit(X_train_cls, y_train_cls)

# ------------------------------------------------------------
# 5. Routing decisions
# ------------------------------------------------------------
true_top1 = np.argmax(Y_test, axis=1)
sorted_preds = np.sort(Y_pred, axis=1)[:, ::-1]
ranked_preds = np.argsort(Y_pred, axis=1)[:, ::-1]

top1_idx = ranked_preds[:, 0]
top2_idx = ranked_preds[:, 1]
top1_score = sorted_preds[:, 0]
top2_score = sorted_preds[:, 1]
gap = top1_score - top2_score

key_to_short = {
    "openai/gpt-4o": "gpt-4o",
    "anthropic/claude-3.5-sonnet": "claude",
    "deepseek/deepseek-chat": "deepseek",
    "perplexity/sonar": "perplexity"
}
llm_names = list(y_scores_df.columns)

# ------------------------------------------------------------
# 6. Evaluate pipeline win rate at different thresholds
# ------------------------------------------------------------
gap_values = [0.07, 0.10, 0.11, 0.12]
pipeline_win_rates = {}

for tau in gap_values:
    print(f"\n=== Pipeline Routing with τ = {tau} ===")
    final_selected_indices = []

    for i in range(len(X_test)):
        emb = X_test[i]
        gap_i = gap[i]

        if gap_i < tau:
            llmA_idx, llmB_idx = top1_idx[i], top2_idx[i]
            llmA_full = llm_names[llmA_idx]
            llmB_full = llm_names[llmB_idx]
            llmA_name = key_to_short[llmA_full]
            llmB_name = key_to_short[llmB_full]

            pair = pd.DataFrame([[llmA_name, llmB_name]], columns=["llm_A", "llm_B"])
            pair_cat = encoder_cls.transform(pair)
            mlp_input = np.hstack([emb, pair_cat[0]])
            prob = mlp.predict_proba([mlp_input])[0][1]
            selected_idx = llmA_idx if prob >= 0.5 else llmB_idx
        else:
            selected_idx = top1_idx[i]

        final_selected_indices.append(selected_idx)

    final_selected_indices = np.array(final_selected_indices)
    chosen_scores = Y_test[np.arange(len(Y_test)), final_selected_indices]

    # --- Compute win rates ---
    win_rates = []
    for llm_idx in range(Y_test.shape[1]):
        spec_mask = (final_selected_indices == llm_idx)
        n_spec = np.sum(spec_mask)
        llm_scores = Y_test[:, llm_idx]
        better_mask = ~spec_mask & (chosen_scores > llm_scores)
        n_better = np.sum(better_mask)
        win_rate = (0.5 * n_spec + n_better) / len(Y_test)
        win_rates.append(win_rate)

    pipeline_win_rates[f"τ={tau}"] = [f"{100 * wr:.2f}%" for wr in win_rates]

# ------------------------------------------------------------
# 7. Print Win Rate Comparison Table
# ------------------------------------------------------------
print("\n=== Pipeline Win Rate Comparison by Threshold ===")
print(f"{'τ':<10} {'GPT-4o':>10} {'Claude':>10} {'DeepSeek':>10} {'Perplexity':>12}")
for tau_label, rates in pipeline_win_rates.items():
    print(f"{tau_label:<10} {rates[0]:>10} {rates[1]:>10} {rates[2]:>10} {rates[3]:>12}")


In [ ]:
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt

# === 1. Load and prepare the test data ===
with open("df_processed.pkl", "rb") as f:
    df_processed = pickle.load(f)

df_raw = pd.read_csv("ranked_responses_final.csv")
prompt_to_source = (
    df_raw[["Prompt", "Source"]]
    .drop_duplicates()
    .set_index("Prompt")["Source"]
)
df_processed["Source"] = df_processed["prompt"].map(prompt_to_source)
df_processed = df_processed.dropna(subset=["Source"]).reset_index(drop=True)

# Features and labels
X = np.vstack(df_processed["embedding"].values)
X_norm = normalize(X, norm="l2", axis=1)

llm_keys = [
    "openai/gpt-4o",
    "anthropic/claude-3.5-sonnet",
    "deepseek/deepseek-chat",
    "perplexity/sonar"
]
y_scores_df = pd.DataFrame(
    df_processed["scores"].tolist(),
    columns=llm_keys
)
Y = y_scores_df.values
category_labels = df_processed["Source"].astype("category").cat.codes

# Train-test split (stratified by Source)
X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm, Y, test_size=0.20, stratify=category_labels, random_state=42
)

# === 2. Train the Random Forest regression model (Router) ===
rf = MultiOutputRegressor(RandomForestRegressor(
    n_estimators=200, max_depth=25, min_samples_leaf=5, random_state=42, n_jobs=-1
))
rf.fit(X_train, Y_train)
Y_pred = rf.predict(X_test)

# === 3. Load or train MLP binary classifier ===
with open("df_pairwise_v2.pkl", "rb") as f:
    df_pairwise = pickle.load(f)

X_embed_cls = np.vstack(df_pairwise["embedding"].values)
encoder_cls = OneHotEncoder(sparse_output=False)
X_cat_cls = encoder_cls.fit_transform(df_pairwise[["llm_A", "llm_B"]])
X_cls = np.hstack([X_embed_cls, X_cat_cls])
y_cls = df_pairwise["label"].values

X_train_cls, X_val_cls, y_train_cls, y_val_cls = train_test_split(
    X_cls, y_cls, test_size=0.2, random_state=42
)
mlp = MLPClassifier(hidden_layer_sizes=(256, 128), max_iter=300, random_state=42)
mlp.fit(X_train_cls, y_train_cls)

# === 4. Confidence-based routing logic with correct mapping ===
true_top1 = np.argmax(Y_test, axis=1)
sorted_preds = np.sort(Y_pred, axis=1)[:, ::-1]
ranked_preds = np.argsort(Y_pred, axis=1)[:, ::-1]

top1_idx = ranked_preds[:, 0]
top2_idx = ranked_preds[:, 1]
top1_score = sorted_preds[:, 0]
top2_score = sorted_preds[:, 1]
gap = top1_score - top2_score

key_to_short = {
    "openai/gpt-4o": "gpt-4o",
    "anthropic/claude-3.5-sonnet": "claude",
    "deepseek/deepseek-chat": "deepseek",
    "perplexity/sonar": "perplexity"
}

thresholds = np.arange(0.01, 0.21, 0.01)
results = []

for tau in thresholds:
    routed_set_contains_best = []
    overall_selection_correct = []

    for i in range(len(X_test)):
        emb = X_test[i]
        routed_llms = [top1_idx[i]]
        use_classifier = gap[i] < tau

        if use_classifier:
            routed_llms.append(top2_idx[i])

        # Coverage accuracy
        routed_set_contains_best.append(true_top1[i] in routed_llms)

        # Final prediction
        if not use_classifier:
            selected_idx = top1_idx[i]
        else:
            llmA_idx, llmB_idx = top1_idx[i], top2_idx[i]
            llmA_full = list(y_scores_df.columns)[llmA_idx]
            llmB_full = list(y_scores_df.columns)[llmB_idx]
            llmA_name = key_to_short[llmA_full]
            llmB_name = key_to_short[llmB_full]
            pair = pd.DataFrame([[llmA_name, llmB_name]], columns=["llm_A", "llm_B"])
            pair_cat = encoder_cls.transform(pair)
            mlp_input = np.hstack([emb, pair_cat[0]])
            prob = mlp.predict_proba([mlp_input])[0][1]
            selected_idx = llmA_idx if prob >= 0.5 else llmB_idx

        overall_selection_correct.append(selected_idx == true_top1[i])

    routed_set_acc = np.mean(routed_set_contains_best)
    overall_acc = np.mean(overall_selection_correct)
    results.append((tau, routed_set_acc, overall_acc))

# Convert to DataFrame
results_df = pd.DataFrame(
    results,
    columns=["Threshold", "Coverage Accuracy", "Overall Selection Accuracy"]
)
print("\n--- Accuracy Results ---")
print(results_df)

# === 5. Compute classifier usage: Pr[g(p) < tau] ===
classifier_usage = []

for tau in thresholds:
    usage_count = np.sum(gap < tau)
    usage_rate = usage_count / len(gap)
    classifier_usage.append((tau, usage_rate))

classifier_usage_df = pd.DataFrame(
    classifier_usage, columns=["Threshold", "Classifier Usage"]
)
print("\n--- Classifier usage (Pr[g < tau]) ---")
print(classifier_usage_df)

# === 6. Plotting ===
plt.figure(figsize=(7, 4))
plt.plot(results_df["Threshold"], results_df["Coverage Accuracy"], marker='o', label="Coverage Accuracy")
plt.plot(results_df["Threshold"], results_df["Overall Selection Accuracy"], marker='s', label="Overall Selection Accuracy")
plt.plot(classifier_usage_df["Threshold"], classifier_usage_df["Classifier Usage"], marker='^', label="Classifier Usage", linestyle='--')
plt.xlabel("Gap Threshold ($\\tau$)")
plt.ylabel("Metric Value")
plt.title("Routing Performance vs. Confidence Threshold")
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()
